In [3]:
import os

from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# API 키 가져오기
MY_OPENAI_KEY = os.getenv("MY_OPENAI_KEY")

client = OpenAI(api_key=MY_OPENAI_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


# 데이터 전처리 함수 작성 241210

### 리뷰 JSON 파일 로드

In [9]:
import json

with open('./resources/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[: 3]

[{'review': '서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍',
  'stars': 4,
  'date': '2024.01.24'},
 {'review': '인사동점 이용해보고 너무 좋아서 용산점 이용했습니다~\n건물도 너무 깨끗하고 직원들도 너무 친절했어요~\n게다가 무인락커가 있어서 락커에 짐보관이 너무 편하고좋았습니다~^^',
  'stars': 5,
  'date': '6일 전'},
 {'review': '친구가 생일이라서 다 같이 방문을 했는데 1

#### 좋은 평점 및 나쁜 평점 정의
 - 좋은 평점 = 별 5개
 - 나쁜 평점 = 별 4개 이하

In [10]:
good_count, bad_count = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_count += 1
    else:
        bad_count += 1

good_count, bad_count

(79, 21)

### 좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [11]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') # 스페셜 토큰 추가
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') # 스페셜 토큰 추가

reviews_bad[:3]

['[REVIEW_START]서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍[REVIEW_END]',
 '[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네

In [13]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:1000]

'[REVIEW_START]서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍[REVIEW_END]\n[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네요.\n

### 전처리 함수 작성

In [15]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./resources/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date # 어차피 6개월 안에 들어와서..

        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:500], bad[:500]

('[REVIEW_START]인사동점 이용해보고 너무 좋아서 용산점 이용했습니다~\n건물도 너무 깨끗하고 직원들도 너무 친절했어요~\n게다가 무인락커가 있어서 락커에 짐보관이 너무 편하고좋았습니다~^^[REVIEW_END]\n[REVIEW_START]수영장이 이쁘고 사진찍기 좋아요. 친절하셔서 좋앗어요[REVIEW_END]\n[REVIEW_START]체크인할때 프론트 여자직원이 정말 친절해서 기억에 남아요. 서울에 또 방문할때 여기 숙소가 1순위입니다. 감사합니다[REVIEW_END]\n[REVIEW_START]깔끔하게 쉬다 옴니다요[REVIEW_END]\n[REVIEW_START]용산역과 가깝고 주변에 아이파크몰도 있어서\n친구들과 하루 날잡아 쉬기 좋았어요.\n베개높이에 민감한데 적당하고 샤워실과 화장실 세면대가 각각 따로 있어서 여자3명이 머물기 편했습니다.\n다음엔 못온 친구와 4인실 이용해보고 싶네요.[REVIEW_END]\n[REVIEW_START]한강뷰가 잘 안보인다는 후기가 있어 걱정했는데',
 '[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네요.\n음식 자체가 퀄리티가 좋아서 또 갈 예정입니다![REVIEW_END]\n[REVIEW_START]3인 이그제큐티브로 이용했는데 티타임, 주류무제한 해피아워, 뷔페식 조식이 다포함이 된 가격이라 넘 좋았습니다!\n객실도 깨끗하고 직원분들도 친절하고, 무엇보다 유료인 수영장이 무료로 포함이 되어 있는 점이 좋았습니다.\n서울에 가본 이그제큐티브 중에서도 제일 음식 퀄리티가 높아서 2번째 방문이고 다음에도 또 갈 계획 있습니다![REVIEW_END]\n[REVIEW_START]깨끗하고 좋앗어요

### 평가용 함수 작성

#### 평가 기준 설정
- MT-Bench 논문 기반 Pariwise Comparison (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수 라는게 애매할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편!

#### 평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accomodations)

In [ ]:
def pairwise_eval(reviews, answer_a, answer_b):
        eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
        
        completion = client.chat.completions.create(
                model='gpt-4o-2024-05-13',
                messages=[{'role': 'user', 'content': eval_prompt}],
                temperature=0.0
        )

        return completion

### Baseline 모델 개발

In [18]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [19]:
reviews, _ = preprocess_reviews(path='./resources/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

다양한 리뷰를 요약하면, 숙소는 깨끗하고 직원들이 친절하며 위치가 좋다는 공통점이 있습니다. 수영장이나 조식 등 시설에 대한 긍정적인 평가도 많이 있습니다. 몇몇 리뷰에서는 주차비가 비싸다는 언급이 있지만 전반적으로 만족스러운 경험을 한다는 내용이 많이 담겨 있습니다.
